In [2]:
import os
import json
import time
import random
from typing import List, Dict, Any, Optional, Literal, TypedDict
from uuid import uuid4
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pinecone import Pinecone
import requests

In [ ]:
# =========================================================
# ENV & CLIENTS (same as previous working version)
# =========================================================
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")  # optional web agent
USE_TAVILY_DEFAULT = os.getenv("USE_TAVILY", "true").lower() in {"1","true","yes","y"}

if not OPENAI_API_KEY:
    raise RuntimeError("Please set OPENAI_API_KEY")
if not PINECONE_API_KEY:
    raise RuntimeError("Please set PINECONE_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)
emb = OpenAIEmbeddings(model="text-embedding-3-large")
llm_fast = ChatOpenAI(model="gpt-5-mini", temperature=0.2)
llm_reason = ChatOpenAI(model="gpt-5-mini", temperature=0.2)

In [4]:
# =========================================================
# INDEX MAP (unchanged)
# =========================================================
INDEXES = {
    "acts": "act-agentic-chunking",
    "rules": "rules-agentic-chunking",
    "sro": "sro-agentic-chunking",
    "ordinances": "ordinance-agentic-chunking",
}

In [5]:
# =========================================================
# DATA MODELS (adds 'web' to Hit.index and hits dict)
# =========================================================
class Hit(BaseModel):
    index: Literal["acts","rules","sro","ordinances","web"]
    id: str
    score: float
    text: str
    metadata: Dict[str, Any] = Field(default_factory=dict)

class RetrievalBundle(BaseModel):
    query: str
    law_name: Optional[str] = None
    law_year: Optional[str] = None
    hierarchy: Dict[str, Optional[str]] = Field(default_factory=lambda: {
        "part": None, "chapter": None, "section": None, "subsection": None, "clause": None
    })
    hits: Dict[str, List[Hit]] = Field(default_factory=lambda: {
        "acts": [], "rules": [], "sro": [], "ordinances": [], "web": []
    })
    merged: List[Hit] = Field(default_factory=list)
    latest_verification: Dict[str, Any] = Field(default_factory=dict)
    final_answer: Optional[str] = None
    citations: List[Dict[str, Any]] = Field(default_factory=list)
    use_web: bool = USE_TAVILY_DEFAULT
    web_query: Optional[str] = None

In [23]:
# =========================================================
# HELPERS (as before)
# =========================================================

def _norm(s: Optional[str]) -> Optional[str]:
    if not s: return s
    return " ".join(str(s).strip().split())


def _build_queries(b: RetrievalBundle) -> List[str]:
    q = [b.query]
    if b.law_name:
        q += [
            f"{b.law_name} {b.law_year or ''}".strip(),
            f"{b.law_name} consolidated version latest amendment",
            f"{b.law_name} Rules Regulations",
            f"{b.law_name} SRO latest supersession",
        ]
    for k in ["part","chapter","section","subsection","clause"]:
        if b.hierarchy.get(k):
            q.append(f"{b.law_name or ''} {k} {b.hierarchy[k]}".strip())
    out, seen = [], set()
    for s in q:
        s = _norm(s)
        if s and s.lower() not in seen:
            out.append(s); seen.add(s.lower())
    return out


def _embed(q: str) -> List[float]:
    return emb.embed_query(q)


def _pc_query(index_name: str, vec: List[float], top_k: int = 10) -> List[Dict[str, Any]]:
    index = pc.Index(index_name)
    res = index.query(vector=vec, top_k=top_k, include_metadata=True)
    return res.get("matches", [])

In [22]:
# =========================================================
# RETRIEVAL (Pinecone) — same logic
# =========================================================

def retrieve_per_index(bundle: RetrievalBundle, top_k: int = 10) -> RetrievalBundle:
    queries = _build_queries(bundle)
    all_hits = {"acts": [], "rules": [], "sro": [], "ordinances": [], "web": bundle.hits.get("web", [])}
    for q in queries:
        vec = _embed(q)
        for tag, idx in INDEXES.items():
            try:
                res = _pc_query(idx, vec, top_k=top_k)
                for m in res:
                    md = m.get("metadata",{}) or {}
                    txt = md.get("page_content") or md.get("text") or ""
                    h = Hit(index=tag, id=m["id"], score=m.get("score",0.0), text=txt, metadata={**md, "_query": q, "_index_name": idx})
                    all_hits[tag].append(h)
            except Exception as e:
                print(f"[WARN] Retrieval failed for {idx}: {e}")
    bundle.hits = all_hits
    return bundle


In [8]:
# =========================================================
# WEB SEARCH (Tavily) — Lawyer-Style staged queries
# =========================================================
TAVILY_ENDPOINT = "https://api.tavily.com/search"

ALLOWED_DOMAINS_DEFAULT = [
    "bdlaws.minlaw.gov.bd",
    "nbr.gov.bd",
    "mof.gov.bd",
    "dpp.gov.bd",
    "cabinet.gov.bd",
    "bangladesh.gov.bd",
]
_extra_domains = os.getenv("TAVILY_ALLOWED_DOMAINS", "").strip()
if _extra_domains:
    for d in _extra_domains.split(","):
        d = d.strip()
        if d and d not in ALLOWED_DOMAINS_DEFAULT:
            ALLOWED_DOMAINS_DEFAULT.append(d)

_BN2EN = str.maketrans("০১২৩৪৫৬৭৮৯", "0123456789")
_EN2BN = str.maketrans("0123456789", "০১২৩৪৫৬৭৮৯")

def to_en(s: str) -> str: return (s or "").translate(_BN2EN)

def to_bn(s: str) -> str: return (s or "").translate(_EN2BN)

KEYWORDS_TRIGGER = ["latest","recent","সর্বশেষ","গেজেট","gazette","SRO","s.r.o","amend","amendment","updated","supersede"]


def should_web_search(bundle: RetrievalBundle) -> bool:
    if not TAVILY_API_KEY: return False
    if not bundle.use_web: return False
    q = (bundle.web_query or bundle.query or "").lower()
    few_hits = sum(len(v) for k,v in bundle.hits.items() if k != "web") < 6
    kw = any(k in q for k in KEYWORDS_TRIGGER)
    return few_hits or kw


def build_web_queries(bundle: RetrievalBundle) -> List[Dict[str, Any]]:
    q = bundle.query or ""
    law = bundle.law_name or ""
    year = (bundle.law_year or "").strip()
    sec = bundle.hierarchy.get("section") or ""
    subsec = bundle.hierarchy.get("subsection") or ""
    clause = bundle.hierarchy.get("clause") or ""

    sec_en, subsec_en, clause_en = to_en(sec), to_en(subsec), to_en(clause)
    sec_bn, subsec_bn, clause_bn = to_bn(sec_en), to_bn(subsec_en), to_bn(clause_en)

    s1 = [
        {"stage":1, "q": f"{law} {year} official gazette pdf", "domains": ALLOWED_DOMAINS_DEFAULT},
        {"stage":1, "q": f"{law} {year} act text site:bdlaws.minlaw.gov.bd", "domains":["bdlaws.minlaw.gov.bd"]},
    ]

    s2 = []
    if sec or subsec or clause:
        base = f"{law} {year} Section {sec_en}{subsec_en}{clause_en}".strip()
        base_bn = f"{law} {year} ধারা {sec_bn}{subsec_bn}{clause_bn}".strip()
        s2 += [
            {"stage":2, "q": base + " site:bdlaws.minlaw.gov.bd", "domains":["bdlaws.minlaw.gov.bd"]},
            {"stage":2, "q": base_bn + " site:bdlaws.minlaw.gov.bd", "domains":["bdlaws.minlaw.gov.bd"]},
        ]

    s3 = [
        {"stage":3, "q": f"{law} {year} Rules pdf gazette", "domains":["nbr.gov.bd","dpp.gov.bd","mof.gov.bd","bangladesh.gov.bd"]},
        {"stage":3, "q": f"Income Tax Rules {to_en(year) or '2023'} pdf site:nbr.gov.bd", "domains":["nbr.gov.bd"]},
    ]

    s4 = [
        {"stage":4, "q": f"SRO {sec_en or q} {year} site:nbr.gov.bd", "domains":["nbr.gov.bd"]},
        {"stage":4, "q": f"S.R.O {sec_en or q} {year} gazette pdf", "domains":["nbr.gov.bd","dpp.gov.bd","mof.gov.bd"]},
    ]

    s5 = [
        {"stage":5, "q": f"{law} {year} consolidated updated latest amendment site:bdlaws.minlaw.gov.bd", "domains":["bdlaws.minlaw.gov.bd"]},
        {"stage":5, "q": f"{law} {year} latest SRO supersedes site:nbr.gov.bd", "domains":["nbr.gov.bd"]},
        {"stage":5, "q": f"{law} {year} latest gazette publication", "domains":["dpp.gov.bd","nbr.gov.bd","mof.gov.bd","cabinet.gov.bd"]},
    ]

    fallback = [{"stage":9, "q": q, "domains": ALLOWED_DOMAINS_DEFAULT}]

    queries = s1 + s2 + s3 + s4 + s5 + fallback
    out = []
    for spec in queries:
        qq = " ".join(x for x in spec["q"].split() if x)
        if qq:
            out.append({"stage": spec["stage"], "q": qq, "domains": spec["domains"]})
    return out


def tavily_search(query: str, max_results: int = 5, domains: Optional[List[str]] = None) -> List[Dict[str, Any]]:
    headers = {"Content-Type": "application/json"}
    payload = {
        "api_key": TAVILY_API_KEY,
        "query": query,
        "search_depth": "basic",
        "max_results": max_results,
        "include_answer": False,
        "include_images": False,
        "include_domains": domains or ALLOWED_DOMAINS_DEFAULT,
        "exclude_domains": [],
    }
    for attempt in range(3):
        try:
            r = requests.post("https://api.tavily.com/search", headers=headers, data=json.dumps(payload), timeout=15)
            if r.status_code == 200:
                return r.json().get("results", [])
            time.sleep(1.2 * (2 ** attempt) + random.random())
        except Exception:
            time.sleep(1.2 * (2 ** attempt) + random.random())
    return []


def web_search_node(state: Dict[str, Any]) -> Dict[str, Any]:
    bundle: RetrievalBundle = state["bundle"]
    if not should_web_search(bundle):
        return state

    specs = build_web_queries(bundle)
    seen_urls = set()
    web_hits: List[Hit] = []

    for spec in specs:
        if len(web_hits) >= 10:
            break
        results = tavily_search(spec["q"], max_results=4, domains=spec.get("domains"))
        for i, r in enumerate(results):
            url = r.get("url")
            if not url or url in seen_urls:
                continue
            seen_urls.add(url)
            title = r.get("title")
            snippet = r.get("content") or r.get("snippet") or ""
            score = 0.4 + (6 - min(spec["stage"], 6)) * 0.08 + (4 - min(i, 4)) * 0.02
            web_hits.append(
                Hit(
                    index="web",
                    id=url,
                    score=score,
                    text=snippet[:1800],
                    metadata={"title": title, "source_url": url, "_query": spec["q"], "provider": "tavily", "stage": spec["stage"]},
                )
            )

    bundle.hits.setdefault("web", [])
    bundle.hits["web"].extend(web_hits)
    state["bundle"] = bundle
    return state

In [9]:

# =========================================================
# RERANK (unchanged logic)
# =========================================================

def _l2r_rerank(hits: List[Hit]) -> List[Hit]:
    if not hits: return []
    items = "\n\n".join([f"[{i}] (index={h.index}, score={h.score:.4f})\nMETA={h.metadata}\nTEXT={h.text[:1000]}" for i,h in enumerate(hits)])
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Score each item 0-100 for answering the legal query. Prefer exact law+year, hierarchy match, latest SRO/amendments, official Gazette sources. Web is supportive only. I want a clean well organized response. NO '#' sign in the response"),
        ("human", "User query: {query}\n\nItems:\n{items}\n\nReturn JSON: {\"rank\":[{\"i\":<int>,\"score\":<int>}]}" )
    ])
    try:
        resp = (prompt | llm_fast | StrOutputParser()).invoke({"query": hits[0].metadata.get("_query",""), "items": items})
        j = json.loads(resp)
        order = {e.get("i"): e.get("score", 0) for e in j.get("rank",[]) if isinstance(e.get("i"), int)}
        ranked = sorted(range(len(hits)), key=lambda i: order.get(i, 0), reverse=True)
        return [hits[i] for i in ranked]
    except Exception:
        return sorted(hits, key=lambda h: h.score, reverse=True)


def merge_and_rerank(bundle: RetrievalBundle) -> RetrievalBundle:
    merged = []
    for lst in bundle.hits.values():
        merged.extend(lst)
    seen = set(); uniq = []
    for h in merged:
        key = (h.index, h.id)
        if key in seen: continue
        seen.add(key); uniq.append(h)
    bundle.merged = _l2r_rerank(uniq)[:20]
    return bundle

In [10]:
# =========================================================
# ENTITY RESOLVER (unchanged)
# =========================================================
resolver_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract law research entities from the user query in JSON."),
    ("human", "User query:\n{query}\n\nReturn JSON with law_name, law_year, hierarchy {{part, chapter, section, subsection, clause}}")
])


def resolve_entities(bundle: RetrievalBundle) -> RetrievalBundle:
    resp = (resolver_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0) | StrOutputParser()).invoke({"query": bundle.query})
    try:
        j = json.loads(resp)
    except Exception:
        j = {}
    bundle.law_name = _norm(j.get("law_name"))
    bundle.law_year = _norm(j.get("law_year"))
    if "hierarchy" in j and isinstance(j["hierarchy"], dict):
        for k,v in j["hierarchy"].items():
            bundle.hierarchy[k] = _norm(v)
    return bundle


In [24]:
# =========================================================
# VERIFICATION (unchanged prompt, can see web items too)
# =========================================================
ver_prompt = ChatPromptTemplate.from_messages([
    ("system", "Verify latest legal effect given retrieved snippets and metadata from Acts/Rules/SROs; web sources are supportive unless linking to primary gazettes."),
    ("human", "Query:\n{query}\nItems:\n{items}\nReturn JSON with latest_sro_ids, consolidated_effect_asof, notes, reasoning")
])

def verify_amendments(bundle: RetrievalBundle) -> RetrievalBundle:
    if not bundle.merged:
        bundle.latest_verification = {"latest_sro_ids": [], "reasoning": "no hits", "consolidated_effect_asof": None, "notes":[]}
        return bundle
    items = "\n\n".join([f"[{i}] index={h.index} id={h.id} score={h.score:.4f}\nMETA={h.metadata}\nTEXT={h.text[:1200]}" for i,h in enumerate(bundle.merged[:18])])
    resp = (ver_prompt | llm_reason | StrOutputParser()).invoke({"query": bundle.query, "items": items})
    try:
        j = json.loads(resp)
    except Exception:
        j = {"latest_sro_ids": [], "reasoning": "parse_error", "consolidated_effect_asof": None, "notes":[]}
    bundle.latest_verification = j
    return bundle

In [25]:
# =========================================================
# ANSWER (unchanged + includes web citations url)
# =========================================================
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bilingual Bangla+English legal assistant. Cite multiple sources and reflect latest SROs."),
    ("human", "Query:\n{query}\nCtx:\n{ctx}\nVer:\n{ver}\nWrite final answer with Bangla bullets, legal path, latest status, citations (include url if web). Be friendly and professional. Answer must be in great details. Initially explain the problem by providing proper definition with references and sources")
])

def synthesize(bundle: RetrievalBundle) -> RetrievalBundle:
    ctx = "\n\n".join([f"[{i}] index={h.index} id={h.id}\nMETA={h.metadata}\nEXTRACT={h.text[:800]}" for i,h in enumerate(bundle.merged[:12])])
    ver = bundle.latest_verification
    resp = (answer_prompt | llm_reason | StrOutputParser()).invoke({"query": bundle.query, "ctx": ctx or "No context.", "ver": ver})
    bundle.final_answer = resp
    cits = []
    for h in bundle.merged[:12]:
        cits.append({
            "index": h.index,
            "id": h.id,
            "title": h.metadata.get("title") or h.metadata.get("rules_name") or h.metadata.get("act_name") or h.metadata.get("sro_title"),
            "date": h.metadata.get("sro_date") or h.metadata.get("date") or h.metadata.get("gazette_date"),
            "score": h.score,
            "url": h.metadata.get("source_url"),
        })
    bundle.citations = cits
    return bundle

In [15]:
# =========================================================
# LANGGRAPH WIRING (add web node between retrieve and merge)
# =========================================================
class GraphState(TypedDict):
    bundle: RetrievalBundle

def _start(state: GraphState) -> GraphState: return state

def _entities(state: GraphState) -> GraphState:
    state["bundle"] = resolve_entities(state["bundle"]); return state

def _retrieve(state: GraphState) -> GraphState:
    state["bundle"] = retrieve_per_index(state["bundle"], top_k=6); return state

def _web(state: GraphState) -> GraphState:
    return web_search_node(state)

def _merge(state: GraphState) -> GraphState:
    state["bundle"] = merge_and_rerank(state["bundle"]); return state

def _verify(state: GraphState) -> GraphState:
    state["bundle"] = verify_amendments(state["bundle"]); return state

def _answer(state: GraphState) -> GraphState:
    state["bundle"] = synthesize(state["bundle"]); return state

graph = StateGraph(GraphState)
graph.add_node("start", _start)
graph.add_node("entities", _entities)
graph.add_node("retrieve", _retrieve)
graph.add_node("web", _web)
graph.add_node("merge", _merge)
graph.add_node("verify", _verify)
graph.add_node("answer", _answer)

graph.set_entry_point("start")
graph.add_edge("start","entities")
graph.add_edge("entities","retrieve")
graph.add_edge("retrieve","web")
graph.add_edge("web","merge")
graph.add_edge("merge","verify")
graph.add_edge("verify","answer")
graph.add_edge("answer", END)

memory = MemorySaver()
app = graph.compile(checkpointer=memory)

# Safe ensure for out-of-order exec

def _ensure_app():
    global app, memory
    try:
        _ = app
    except NameError:
        memory = MemorySaver()
        app = graph.compile(checkpointer=memory)
    return app

In [16]:
# =========================================================
# PUBLIC RUNNER (same signature + web controls)
# =========================================================

def run_legal_query(user_query: str, law_name: Optional[str]=None, law_year: Optional[str]=None, hierarchy: Optional[Dict[str, Optional[str]]]=None, thread_id: Optional[str]=None, use_web: Optional[bool]=None, web_query: Optional[str]=None) -> RetrievalBundle:
    b = RetrievalBundle(query=user_query)
    if law_name: b.law_name = law_name
    if law_year: b.law_year = law_year
    if hierarchy: b.hierarchy.update(hierarchy)
    if use_web is not None: b.use_web = bool(use_web)
    if web_query: b.web_query = web_query
    cfg = {"configurable": {"thread_id": thread_id or str(uuid4())}}
    _ensure_app()
    out = app.invoke({"bundle": b}, config=cfg)
    return out["bundle"]

In [59]:
out = run_legal_query(
    "আয়কর আইন, ২০২৩ এর ধারা ৫২(১)(ক) অনুযায়ী TDS এর Rules এবং সর্বশেষ SRO কী?",
    use_web=True
)
print(out.final_answer)
print(out.citations)


আদরণীয় ব্যবহারকারী,

আপনার প্রশ্ন অনুযায়ী, আয়কর আইন, ২০২৩ এর ধারা ৫২(১)(ক) অনুযায়ী TDS (উৎসে কর) সম্পর্কিত বিধিমালা এবং সর্বশেষ সংশোধনী (SRO) সম্পর্কে বিস্তারিত তথ্য নিচে উপস্থাপন করা হলো।

---

### ১. আইনি ভিত্তি ও বিধিমালা

- **আইন:** আয়কর আইন, ২০২৩ (২০২৩ সনের ১২ নং আইন)
- **ধারা:** ধারা ৫২(১)(ক) অনুযায়ী উৎসে কর (TDS) ধার্য ও সংগ্রহের বিধান
- **বিধিমালা:** জাতীয় রাজস্ব বোর্ড কর্তৃক ধারা ৩৪৩ এর ক্ষমতাবলে প্রণীত **"উৎসে কর বিধিমালা, ২০২৪"** (Withholding Tax Rules, 2024)

---

### ২. সর্বশেষ সংশোধনী (SRO)

- **SRO নম্বর:** ২৪৮-আইন/আয়কর-৪২/২০২৪
- **প্রকাশের তারিখ:** ১৩ আষাঢ়, ১৪৩১ বঙ্গাব্দ (২৭ জুন, ২০২৪ খ্রিষ্টাব্দ)
- **কার্যকর তারিখ:** ১ জুলাই, ২০২৪
- **প্রকাশক:** জাতীয় রাজস্ব বোর্ড, অর্থ মন্ত্রণালয়, বাংলাদেশ সরকার
- **মূল বিষয়বস্তু:**  
  - উৎসে কর বিধিমালার বিধি ৩ এর সারণী প্রতিস্থাপন, যেখানে বিভিন্ন পণ্যের জন্য TDS হার নির্ধারণ করা হয়েছে।  
  - উদাহরণস্বরূপ:  
    - এমএস বিলেট উৎপাদনে নিয়োজিত শিল্প প্রতিষ্ঠানের ক্ষেত্রে ০.৫%  
    - পেট্রোলিয়াম তেল সরবরাহে ০.৬%  
    - 

In [60]:
out = run_legal_query(
    "Is there any option to get refund of my excess paid tax? Explain in details",
    use_web=True
)
print(out.final_answer)
print(out.citations)

আপনার অতিরিক্ত পরিশোধিত কর ফেরতের বিষয়ে বিস্তারিত ব্যাখ্যা নিচে প্রদান করা হলো:

---

### ১. কর ফেরতের আইনি সুযোগ ও বিধানসমূহ

- **আয়কর আইন, ২০২৩ এর ধারা ২২৪** অনুযায়ী, যদি কোনো ব্যক্তি উপকর কমিশনারকে সন্তুষ্ট করতে পারেন যে, কোনো করবছরে তিনি বা তার পক্ষে পরিশোধিত করের পরিমাণ তার করদায়িত্বের চেয়ে বেশি হয়েছে, তবে তিনি অতিরিক্ত পরিশোধিত কর ফেরত পাওয়ার অধিকারী হবেন।  
- এই ধারা স্পষ্টভাবে করদাতাদের অতিরিক্ত কর ফেরত পাওয়ার অধিকার নিশ্চিত করে।  
- **ধারা ২২৫** অনুসারে, কর ফেরতের দাবির ক্ষেত্রে যদি করদাতার উপর কোনো বকেয়া অর্থ থাকে, তবে সেই বকেয়া অর্থ ফেরতের আগে সমন্বয় করা হবে। অর্থাৎ, বকেয়া থাকলে সেটি ফেরতের টাকা থেকে কর্তন করা হবে।

---

### ২. উৎসে কর (TDS) সম্পর্কিত বিধান ও সনদপত্র

- **উৎসে কর বিধিমালা, ২০২৪** (TDS Rules, 2024) অনুযায়ী, উৎসে কর কর্তন ও জমাদানের নিয়মাবলী নির্ধারিত হয়েছে।  
- **বিধি ১১(১)** অনুযায়ী চাকরি থেকে আয় হতে উৎসে কর কর্তনের সনদপত্র প্রদান করা হয়, যা কর পরিশোধের প্রমাণ হিসেবে ব্যবহৃত হয়।  
- কর কর্তনকারী কর্তৃপক্ষ কর্তৃক প্রদত্ত এই সনদপত্র করদাতাকে

In [56]:
out = run_legal_query(
    "How can I reduce my income tax?",
    use_web=True
)
print(out.final_answer)
print(out.citations)

আপনার আয়কর কমানোর জন্য বাংলাদেশের বর্তমান আইন ও সংশোধিত বিধিমালা অনুসারে প্রধানত নিম্নলিখিত উপায়সমূহ গ্রহণ করতে পারেন:

---

### আয়কর কমানোর উপায়সমূহ (বাংলায়):

- **১. অনুমোদিত দানে কর বিয়োজন (Deduction for Approved Donations):**  
  আয়কর আইন, ২০২৩ এর ষষ্ঠ তফসিল (ধারা ৭৬ দ্রষ্টব্য) অনুযায়ী, আপনি যদি ব্যাংক ট্রান্সফারের মাধ্যমে প্রধানমন্ত্রীর শিক্ষা সহায়তা ট্রাস্ট, সরকারি অনুমোদিত বালিকা বিদ্যালয়, মহিলা কলেজ, কারিগরি প্রশিক্ষণ প্রতিষ্ঠান অথবা কৃষি, বিজ্ঞান, প্রযুক্তি ও শিল্প উন্নয়নের জন্য গবেষণা ও উন্নয়নের সাথে সম্পৃক্ত জাতীয় পর্যায়ের প্রতিষ্ঠানে দান করেন, তবে সেই দানের পরিমাণ আপনার মোট আয় থেকে বিয়োজনযোগ্য হবে।  
  - কোম্পানির ক্ষেত্রে সর্বোচ্চ ১০% বা ৮ কোটি টাকা (যা কম) পর্যন্ত দান কর বিয়োজনযোগ্য।  
  - অন্যান্য করদাতাদের ক্ষেত্রে সর্বোচ্চ ১০% বা ১ কোটি টাকা (যা কম) পর্যন্ত।  
  - ২০২৪ সালের SRO 340 (৯ অক্টোবর ২০২৪) অনুযায়ী, As-Sunnah Foundation-এ দানও এই সুবিধার আওতায় এসেছে।  
  - **আইনি উৎস:** আয়কর আইন, ২০২৩, অংশ ২, ধারা ৭৬; SRO 340-আইন/আয়কর-৪৮/২০২৪  
  - **সূত্র

In [44]:
out = run_legal_query(
    "Please provide the list of exempted income for individuals. Tell me in details",
    use_web=True
)
print(out.final_answer)
print(out.citations)

বাংলাদেশের আয়কর আইন, ২০২৩ (Income Tax Act, 2023) এবং সর্বশেষ সংশোধনসমূহ (বিশেষ করে এসআরও ৩৪০/২০২৪, ৯ অক্টোবর ২০২৪ তারিখে জারি) অনুযায়ী, ব্যক্তিদের জন্য যে আয়সমূহ করমুক্ত বা exempted income হিসেবে গণ্য হয়, তা নিম্নরূপ বিস্তারিতভাবে উপস্থাপন করা হলো:

---

### ব্যক্তিদের জন্য করমুক্ত আয়ের তালিকা (Exempted Income for Individuals):

১. **আন্তর্জাতিক ও আন্তঃসরকারি সংস্থার আয়**  
   - যেকোনো আন্তর্জাতিক বা আন্তঃসরকারি সংস্থা এবং তাদের কর্মচারীদের আয়, যা আইন বা সরকারী চুক্তি অনুযায়ী করমুক্ত।  
   - যেমন: জাতিসংঘ, বিশ্বব্যাংক ইত্যাদি সংস্থার কর্মচারীদের বেতন ও অন্যান্য আয়।  
   - **আইনগত উৎস:** Income Tax Act, 2023, Section 76 এবং Sixth Schedule।

২. **বিদেশী কূটনীতিক ও সরকারী প্রতিনিধি বেতন-ভাতা**  
   - বিদেশী রাষ্ট্রের কূটনীতিক, দূতাবাসের কর্মকর্তা ও নির্দিষ্ট সরকারী প্রতিনিধি যারা বাংলাদেশ সরকারের অনুমোদিত, তাদের বেতন ও ভাতা করমুক্ত।  
   - **আইনগত উৎস:** Income Tax Act, 2023, সংশ্লিষ্ট ধারা ও Sixth Schedule।

৩. **অনুমোদিত দাতব্য ও শিক্ষা প্রতিষ্ঠানসমূহে দান**  
   - করদাতারা ব্য

In [20]:
out = run_legal_query(
    "ক্লায়েন্ট আমার কাছে রির্টান দাখিলের প্রমান চাচ্ছে এখন আমি কি করবো?",
    use_web=True
)
print(out.final_answer)
print(out.citations)

আপনার ক্লায়েন্ট রিটার্ন দাখিলের প্রমাণ চাচ্ছে, এ ক্ষেত্রে আপনি কী করবেন—এ বিষয়ে বিস্তারিত ব্যাখ্যা ও আইনি নির্দেশনা নিচে দেওয়া হলো।

---

### সমস্যা ও প্রাসঙ্গিক সংজ্ঞা

**রিটার্ন দাখিলের প্রমাণ** বলতে বোঝায় করদাতা কর্তৃক আয়কর রিটার্ন জমা দেওয়ার যে নথিপত্র বা প্রমাণাদি, যা করদাতার ব্যবসা বা আয়ের স্থানে সহজে প্রদর্শনযোগ্য হয়। এটি করদাতার রিটার্ন দাখিলের বাধ্যবাধকতা পূরণের প্রমাণ হিসেবে ব্যবহৃত হয়।

বাংলাদেশের **আয়কর আইন, ২০২৩** এর ধারা ২৬৫ অনুযায়ী:

- করদাতাদের রিটার্ন দাখিলের প্রমাণ তাদের ব্যবসার স্থানে সহজে দৃষ্টিগোচর হয় এমন স্থানে প্রদর্শন করতে হয়।
- যদি করদাতা এই বিধান পালন না করেন, তাহলে উপকর কমিশনার ৫,০০০ থেকে ২০,০০০ টাকা পর্যন্ত জরিমানা আরোপ করতে পারেন।  
(সূত্র: আয়কর আইন, ২০২৩, ধারা ২৬৫, [source](#))

---

### করদাতার কর রিটার্ন দাখিলের প্রমাণ প্রদানের উপায়

১. **ব্যবসার স্থানে রিটার্নের কপি প্রদর্শন:**

   - করদাতা তার ব্যবসার স্থানে আয়কর রিটার্নের কপি বা সংশ্লিষ্ট নথি সহজে প্রদর্শন করবেন, যাতে কর কর্তৃপক্ষ বা ক্লায়েন্ট প্রমাণ হিসেবে দেখতে পারেন।  
   - এটি আইনানুগ বাধ্যবাধকতা 

In [26]:
out = run_legal_query(
    "ইভেন্ট ম্যানেজমেন্ট সেবার কমিশন ফীর উপর টি ডি এস রেট কতো?",
    use_web=True
)
print(out.final_answer)
print(out.citations)

ইভেন্ট ম্যানেজমেন্ট সেবার কমিশন ফীর উপর উৎসে কর (TDS) হার সম্পর্কে বিস্তারিত:

---

### ১. সমস্যা ও প্রাসঙ্গিক সংজ্ঞা

**টি ডি এস (TDS)** বা উৎসে কর হলো এমন একটি কর ব্যবস্থা যেখানে কোনো অর্থ পরিশোধের সময়ই কর কর্তন করে সরকারকে জমা দিতে হয়। এটি কর আদায়ের একটি কার্যকর পদ্ধতি, যা আয়কর আইন ও সংশ্লিষ্ট বিধিমালা দ্বারা নিয়ন্ত্রিত।

**ইভেন্ট ম্যানেজমেন্ট সেবা** বলতে বোঝায় বিভিন্ন ধরণের ইভেন্ট (যেমন: সম্মেলন, প্রদর্শনী, সাংস্কৃতিক অনুষ্ঠান, কর্পোরেট ইভেন্ট ইত্যাদি) পরিচালনা ও ব্যবস্থাপনার সেবা। এই সেবার জন্য কমিশন ফি বা পারিশ্রমিক প্রদান করা হলে, সেই অর্থের উপর উৎসে কর কর্তন বাধ্যতামূলক।

---

### ২. আইনি উৎস ও বিধিমালা

- **উৎসে কর বিধিমালা, ২০২৪ (TDS Rules, 2024)** — সর্বশেষ সংশোধিত (২ জুন ২০২৪) বিধিমালা অনুযায়ী, বিধি ৫ এর সারণীতে ইভেন্ট ম্যানেজমেন্টসহ ব্যবস্থাপনা সেবার কমিশন ফীর উপর ২০% হার দিয়ে উৎসে কর কর্তন করতে হবে।  
  - **সূত্র:**  
    - TDS Rules, 2024, বিধি ৫, সারণী-৫-১  
    - [Rules\12. TDS Rules, 2024 (Amendment 02 June 25)_complete_transcription.txt](#) (দেখুন চাঙ্ক আইডি:

In [ ]:
out = run_legal_query(
    "",
    use_web=True
)
print(out.final_answer)
print(out.citations)

In [54]:
out = run_legal_query(
    "How can I check whether my employer is deducting excess income tax or not?",
    use_web=True
)
print(out.final_answer)
print(out.citations)

আপনি কীভাবে যাচাই করবেন যে আপনার নিয়োগকর্তা (employer) আপনার আয় থেকে অতিরিক্ত আয়কর কর্তন করছে কিনা, সে বিষয়ে বিস্তারিত নির্দেশনা নিচে দেওয়া হলো:

---

### ১. উৎসে কর কর্তনের সনদপত্র (TDS Certificate) সংগ্রহ করুন
- **তফসিল ২ অনুযায়ী** (TDS Rules, ২০২৪, বিধি ১১(১)) আপনার নিয়োগকর্তা কর্তৃক কর্তিত আয়কর সম্পর্কিত একটি **“উৎসে কর কর্তন/সংগ্রহের সনদপত্র”** প্রদান করা বাধ্যতামূলক।  
- এই সনদপত্রে থাকবে:  
  - নিয়োগকর্তার নাম ও ঠিকানা  
  - আপনার নাম, পদবী ও টিআইএন (TIN)  
  - বেতন, ভাতা ও অন্যান্য অর্থ প্রদানের বিস্তারিত  
  - কর্তিত করের পরিমাণ  
  - কর সরকারী কোষাগারে জমা দেওয়ার তথ্য  
- সনদপত্র পাওয়ার মাধ্যমে আপনি জানতে পারবেন কত টাকা কর কর্তন হয়েছে এবং তা সরকারে জমা হয়েছে কিনা।  
- **সূত্র:** [TDS Rules, 2024 - তফসিল ২ (বিধি ১১(১))](https://nbr.gov.bd)  

---

### ২. আপনার আয়ের উপর ভিত্তি করে কর নিরূপণ করুন
- আপনার মোট আয় ও করযোগ্য আয় নিরূপণ করুন, যা আয়কর আইন, ২০২৩ এর ধারা ২৬ অনুযায়ী নির্ধারিত হয়।  
- আয়কর আইন, ২০২৩ অনুযায়ী আপনার আয়ের উৎস, করযোগ্য আয় ও কর হার সম্পর্ক

In [55]:
out = run_legal_query(
    "What is the tax rate for capital gain for individuals?",
    use_web=True
)
print(out.final_answer)
print(out.citations)

আপনার প্রশ্ন ছিল: **ব্যক্তিদের জন্য মূলধন লাভ (Capital Gain) করের হার কত?**

### সংক্ষিপ্ত উত্তর:
বর্তমানে প্রাপ্ত আইন ও বিধিমালা অনুসারে, বাংলাদেশে ব্যক্তিদের জন্য **মূলধন লাভ করের নির্দিষ্ট হার সরাসরি আইন বা বিধিমালায় স্পষ্টভাবে উল্লেখিত নেই**। তবে, আয়কর আইন, ২০২৩ এবং পরবর্তী ফাইন্যান্স অধ্যাদেশ ও জাতীয় রাজস্ব বোর্ডের (NBR) উৎসে কর বিধিমালা অনুযায়ী ব্যক্তিদের করযোগ্য আয়ের উপর সারচার্জের হার এবং বিভিন্ন উৎস থেকে আয়ের উপর উৎসে কর (TDS) হার নির্ধারিত আছে, যা মূলধন লাভ করের হার নয় বরং আয়ের উপর প্রযোজ্য অন্যান্য করের হার।

---

### বিস্তারিত ব্যাখ্যা ও প্রাসঙ্গিক আইন:

১. **আয়কর আইন, ২০২৩ (Income Tax Act, 2023)**  
   - আইন অনুযায়ী, ব্যক্তির মোট আয়ের মধ্যে বিভিন্ন উৎস থেকে প্রাপ্ত আয় অন্তর্ভুক্ত হয় (ধারা ২৬)।  
   - কিন্তু মূলধন লাভ করের নির্দিষ্ট হার বা বিধান সরাসরি উল্লিখিত হয়নি।  
   - [আয়কর আইন, ২০২৩ - ধারা ২৬](https://www.bgpress.gov.bd) (উল্লেখযোগ্য অংশ সংক্ষেপে উপরে দেওয়া হয়েছে)

২. **ফাইন্যান্স অধ্যাদেশ, ২০২৫ (Finance Ordinance, 2025)**  
   - ১ জুলাই ২০২৬ থেকে কা